# Tutorial 2: Generating Age Probability Distributions for Stars with Gaia Data

In the previous tutorial, we've calibrated the age--action spline model using a calibration sample of red giant branch stars with independent, asteroseismic ages. 

Now, let's use that spline model to generate age probabilities for other stars.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import zoomies

%load_ext autoreload
%autoreload 2

## Query Gaia

You will need to download the Gaia DR3 data for any stars you're interested in. You can perform a mass query at https://gea.esac.esa.int/archive/ and read in the resulting table, or you can use ``astroquery`` to download the data right from your notebook, as we'll do here:

In [2]:
# Install astroquery if you haven't already!
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

In [ ]:
# Do your Gaia query or input a table of Gaia data

import astropy.units as u
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=280, dec=60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.5, u.deg)
height = u.Quantity(0.5, u.deg)

r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

In [ ]:
# Filtering out stars without radial velocity
r = r[~r['radial_velocity'].mask]

Here's the Gaia data for the couple of stars we're interested in, as an astropy table:

In [ ]:
r

## Calculate Jz

You now need to calculate ln(Jz) (log of vertical action) for each Gaia star. The star must have a radial velocity. (Make sure you check the number of stars that have radial velocities from Gaia for the sample you're interested in.)

TODO: You must input a table with at least two rows. It doesn't work if you just input one row of Gaia data.

In [ ]:
# This function returns a table with three new 
# columns for the action angles at the end: Jz, Jphi, and Jtheta

# This line may take a while for very large tables.

r_actions = zoomies.calc_jz(r)
r_actions

## Age-Jz Relation Calibrated on RGB ages

We previously calibrated an age--vertical action spline model in Tutorial 1, and saved it in the ``"RGB_spline_model"`` directory. Let's read that model back in here.

Alternatively, you can download the folder from Zenodo: 10.5281/zenodo.10048927

In [ ]:
spline_direct = "../RGB_spline_model/"

# Reading the pre-calibrated spline model back in
RGB_spline = zoomies.read(directory=spline_direct)

The ``KinematicAgeSpline.evaluate_ages()`` function evaluates dynamical ages for the ln(Jz) values in our table.

In [ ]:
# this function evaluates age predictions for a set of ln(Jz) values.

eval_grid, starRGB_eval_pdf = RGB_spline.evaluate_ages(np.log(r_actions['Jz']), eval_grid=np.linspace(0, 14, 1000))

This is the age probabiltiy distribution for the first star in our Gaia table:

In [ ]:
plt.plot(eval_grid, starRGB_eval_pdf[0])

plt.xlabel('Age gyr')
plt.ylabel('PDF')
plt.title('lnJz = ' + str((np.log(r_actions['Jz'][0]))))

And here it is for the second star:

In [ ]:
# this is the age prediction for the second star in the table

plt.plot(eval_grid, starRGB_eval_pdf[1])

plt.xlabel('Age gyr')
plt.ylabel('PDF')

plt.title('lnJz = ' + str((np.log(r_actions['Jz'][1]))))